In [0]:
%sql
-- 1. Create the External Location object in Databricks
-- This connects the "managed-storage" container to the "ecomm_storage_credential" key
CREATE EXTERNAL LOCATION IF NOT EXISTS ecomm_managed_location
URL 'abfss://managed-storage@ecommstorageaccount.dfs.core.windows.net/'
WITH (
  CREDENTIAL `ecomm_storage_credential` )

In [0]:
%sql
-- Step 2: Create the external location
CREATE EXTERNAL LOCATION IF NOT EXISTS ecomm_managed_location
URL 'abfss://managed-storage@ecommstorageaccount.dfs.core.windows.net/'
WITH (STORAGE CREDENTIAL ecomm_storage_credential);

-- Step 3: Create the catalog using the managed location
CREATE CATALOG IF NOT EXISTS ecomm_data_project
MANAGED LOCATION 'abfss://managed-storage@ecommstorageaccount.dfs.core.windows.net/';

In [0]:
%sql
-- Section 2: Schemas (The Rooms)
CREATE SCHEMA IF NOT EXISTS ecomm_data_project.bronze;
CREATE SCHEMA IF NOT EXISTS ecomm_data_project.silver;
CREATE SCHEMA IF NOT EXISTS ecomm_data_project.gold;
CREATE SCHEMA IF NOT EXISTS ecomm_data_project.audit;

-- Section 3: Volumes (The Storage Folders)
CREATE VOLUME IF NOT EXISTS ecomm_data_project.ecomm_raw.landing_zone;
CREATE VOLUME IF NOT EXISTS ecomm_data_project.bronze.checkpoints;

In [0]:
%sql
-- Now this simple command will work perfectly!
-- No LOCATION path needed. Databricks handles it automatically.
CREATE TABLE IF NOT EXISTS ecomm_data_project.audit.pipeline_logs (
    run_id STRING,
    job_name STRING,
    layer STRING,
    status STRING,
    records_processed LONG,
    timestamp TIMESTAMP,
    error_message STRING
)
USING DELTA;

In [0]:
%sql
-- 1. Wipe the current version
--DROP TABLE IF EXISTS ecomm_data_project.silver.users;

-- 2. Re-create with the MISSING COLUMNS included
CREATE TABLE IF NOT EXISTS ecomm_data_project.silver.users (
    identifierHash STRING NOT NULL,         
    countrycode STRING,
    gender STRING,
    civilitytitle_clean STRING,
    language_full STRING,
    account_age_years DOUBLE,
    -- ADDING THE MISSING COLUMNS HERE:
    productsSold INT,
    productsWished INT,
    -- SCD Type 2 tracking columns
    is_current BOOLEAN,
    effective_start_date TIMESTAMP,
    effective_end_date TIMESTAMP
)
USING DELTA;

-- 3. Re-add the constraints
ALTER TABLE ecomm_data_project.silver.users ADD CONSTRAINT valid_gender CHECK (gender IN ('Male', 'Female', 'Other'));
ALTER TABLE ecomm_data_project.silver.users ADD CONSTRAINT positive_age CHECK (account_age_years >= 0);

In [0]:
%sql
-- 1. Hard Rule: Users must have an ID
-- If a row arrives without identifierHash, the whole batch will fail (Safety First)
ALTER TABLE ecomm_data_project.silver.users 
ADD CONSTRAINT user_id_not_null CHECK (identifierHash IS NOT NULL);

In [0]:
%sql
CREATE OR REPLACE VIEW IF NOT EXISTS ecomm_data_project.gold.secure_users AS
SELECT 
    CASE 
        WHEN is_account_group_member('admin') THEN identifierHash 
        ELSE '***MASKED***' 
    END AS identifierHash,
    countrycode,
    gender,
    language_full,
    is_current
FROM ecomm_data_project.silver.users;